In [1]:
!pip install eli5

    100% |████████████████████████████████| 102kB 21.1MB/s a 0:00:01
    100% |████████████████████████████████| 51kB 19.2MB/s ta 0:00:01
  Running setup.py bdist_wheel for tabulate ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/2b/67/89/414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
Successfully built tabulate
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('./dataset/airbnb_seasons_20172018/airbnb_cleaned_winter.csv')

In [4]:
df

,Unnamed: 0,id,host_id,scrape_id,zipcode,neighbourhood_cleansed,property_type,room_type,accommodates,bedrooms,beds,bed_type,price,number_of_reviews,review_scores_rating,availability_30,minimum_nights,bathrooms,host_is_superhost,host_response_rate
0,0,2515,2758,20181206022948,10026,harlem,1,1.01,3,1.0,2.0,1.00,59.0,175,93.000000,3,2,1.0,1.0,70.0
1,1,21456,42032,20181206022948,10025,upperwestside,1,1.00,2,1.0,1.0,1.00,140.0,76,94.000000,8,5,1.0,0.0,80.0
2,2,2539,2787,20181206022948,11218,kensington,1,1.01,4,1.0,1.0,1.00,149.0,9,98.000000,30,1,1.0,1.0,100.0
3,3,2595,2845,20181206022948,10018,midtown,1,1.00,2,0.0,1.0,1.00,225.0,41,95.000000,15,1,1.0,0.0,82.0
4,4,21644,82685,20181206022948,10031,harlem,1,1.01,1,1.0,1.0,1.00,89.0,1,100.000000,30,1,1.0,0.0,0.0
5,5,3330,4177,20181206022948,11206,williamsburg,1,1.01,2,1.0,1.0,1.00,70.0,38,98.000000,11,5,1.0,0.0,100.0
6,6,21794,83257,20181206022948,10014,chelsea,1,1.01,2,1.0,1.0,1.00,111.0,26,99.000000,24,30,1.0,1.0,100.0
7,7,3647,4632,20181206022948,10027,harlem,1,1.01,2,1.0,1.0,1.02,150.0,0,93.369569,30,3,1.0,0.0,0.0
8,8,22911,87773,20181206022948,11233,bedford-stuyvesant,1,1.00,4,1.0,3.0,1.00,125.0,140,92.000000,0,3,1.0,0.0,100.0
9,9,22918,32294,20181206022948,10031,harlem,1,1.01,2,1.0,1.0,1.00,60.0,10,82.000000,0,1,1.0,0.0,100.0


## Gradient Boosting Machine

In [ ]:
import math
import eli5
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from eli5.sklearn import PermutationImportance

In [ ]:
def LRfindAveragePrice(location):
    return (df[(df.neighbourhood_cleansed == location)])

area = LRfindAveragePrice('woodside')
area.drop(['neighbourhood_cleansed'], axis=1)
area.info()

In [ ]:
median_beds = math.floor(df.beds.median())
median_bedrooms = math.floor(df.bedrooms.median())
median_bathrooms = math.floor(df.bathrooms.median())

area['beds'] = area['beds'].fillna(value=median_beds)
area['bedrooms'] = area['bedrooms'].fillna(value=median_bedrooms)
area['bathrooms'] = area['bathrooms'].fillna(value=median_bathrooms)



In [ ]:
y = area['price']
X = area.drop(['price', 'Unnamed: 0', 'id', 'host_id', 'scrape_id', 'zipcode', 'neighbourhood_cleansed'], axis=1) 
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

In [ ]:
%%time 
regr = GradientBoostingRegressor(n_estimators=32, learning_rate=0.28, max_depth=4, subsample=1.0,
                                 random_state=42)
regr.fit(X_train, y_train)
print(r2_score(y_val, regr.predict(X_val)))

#R^2 score

In [ ]:
perm = PermutationImportance(regr, random_state=42).fit(X_val, y_val)
eli5.show_weights(perm, top=X.shape[1], feature_names = X.columns.tolist())

In [ ]:
area['price'].mean()